In [2]:
from itertools import product as prod, combinations, combinations_with_replacement as cwr
from typing import Callable, Union, Tuple, Set, overload, Optional
from operator import le, lt, ge, gt, ne  # lower-equal, lower-than, greater-equal, greater-than, not-equal
from random import randint
import inspect
from pprint import pformat

powerset = lambda v: {tuple(combo) for r in range(len(v) + 1) for combo in combinations(v, r)}
cartesian_prod = lambda v:prod(v, v) # cartesian_prod({1,2}) = [(1, 1), (1, 2), (2, 1), (2, 2)]
universal_rel = lambda a,b:True
empty_rel = lambda a,b:False
identity_rel = lambda a,b:a==b
opposite_rel = lambda a,b:a==-b

# inverse_rel = lambda a,b:
R = Callable[[int, int], bool]
Prod = Set[Tuple[int, int]] # {<1,2>, <3,4>}
Vec = Set[int] # {1, 2, 3, 4}
RawRelation = Tuple[R, Vec, Optional[Vec]] # ( lambda a,b:... , {1,2,3}, {3,4,5}? )
Relation = Union[
    Prod, 
    RawRelation
    ] 

In [3]:
def is_product(thing):
    """
    Returns True if thing is Prod, i.e. is of the form { <a,b>, <c,d>, ... }. Don't care about exact types (tuples, sets, etc)
    """
    try:
        for x in thing:
            if not isinstance(x, (tuple, list)):
                return False
        return True
            
    except:
        return False

In [4]:
def is_raw_relation(thing):
    """
    Returns True if thing is (lambda, Vec, Vec?)
    """
    try:
        length = len(thing)
        if not isinstance(thing, (tuple, list)) or length < 2 or length > 3:
            return False
        
        maybe_ok = callable(thing[0]) and isinstance(thing[1], (list,tuple,set))
        if length == 3:
            return maybe_ok and isinstance(thing[2], (list,tuple,set))
        else:
            return maybe_ok
        
    except:
        return False

In [5]:
list(cartesian_prod({}))
# empty

[]

In [6]:
list(cartesian_prod({1}))
# (1, 1)

[(1, 1)]

In [7]:
list(cartesian_prod({1,2}))
# (1, 1), (1, 2), 
# (2, 1), (2, 2)

[(1, 1), (1, 2), (2, 1), (2, 2)]

In [8]:
list(cartesian_prod({1,2,3}))
# (1, 1), (1, 2), (1, 3), 
# (2, 1), (2, 2), (2, 3), 
# (3, 1), (3, 2), (3, 3)

[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]

In [9]:
list(cartesian_prod({1,2,3,4}))
# (1, 1), (1, 2), (1, 3), (1, 4),
# (2, 1), (2, 2), (2, 3), (2, 4),
# (3, 1), (3, 2), (3, 3), (3, 4),
# (4, 1), (4, 2), (4, 3), (4, 4)

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

In [10]:
powerset({1,2,3,4})
# (),
# (1,), (2,), (3,), (4,)
# (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4),
# (1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4),
# (1, 2, 3, 4)

{(),
 (1,),
 (1, 2),
 (1, 2, 3),
 (1, 2, 3, 4),
 (1, 2, 4),
 (1, 3),
 (1, 3, 4),
 (1, 4),
 (2,),
 (2, 3),
 (2, 3, 4),
 (2, 4),
 (3,),
 (3, 4),
 (4,)}

In [11]:
def partition_to_equiv_classes(prod:Prod)->Set[Prod]:
    """
    >>> partition = { (1,2,), (3,) }
    >>> partition_to_equiv_classes(partition)
    {(1, 1), (1, 2), (2, 1), (2, 2), (3, 3)}
    """
    if not is_product(prod):
        raise ValueError(f"partition_to_equiv_classes(prod) expecting a 2D+ structure, got {pformat(prod)}")
    equiv_classes = set()
    for subset in prod:
        for x in cartesian_prod(subset):
            equiv_classes.add(x)
    return equiv_classes
    

In [12]:
A = {1,2,3}
for i in range(1, len(A)+1):
    print()
    combos = list(combinations(A,i))
    if sum(map(len,combos)) != len(A):
        for combo in combos:
            remainder = A-set(combo)
            print(f'{combo=}, {remainder=}')
            partition = {combo, tuple(remainder)}
            equiv_classes = partition_to_equiv_classes(partition)
            print(f'{equiv_classes=}')
    else:
        print(f'{combos=}')
        equiv_classes = partition_to_equiv_classes(combos)
        print(f'{equiv_classes=}')


combos=[(1,), (2,), (3,)]
equiv_classes={(1, 1), (3, 3), (2, 2)}

combo=(1, 2), remainder={3}
equiv_classes={(1, 2), (2, 1), (1, 1), (3, 3), (2, 2)}
combo=(1, 3), remainder={2}
equiv_classes={(3, 1), (1, 1), (3, 3), (2, 2), (1, 3)}
combo=(2, 3), remainder={1}
equiv_classes={(1, 1), (2, 3), (3, 3), (2, 2), (3, 2)}

combos=[(1, 2, 3)]
equiv_classes={(1, 2), (2, 1), (3, 1), (1, 1), (2, 3), (3, 3), (2, 2), (3, 2), (1, 3)}


In [13]:

partition = { (1,2,), (3,) }
partition_to_equiv_classes(partition)

{(1, 1), (1, 2), (2, 1), (2, 2), (3, 3)}

In [14]:
relR = lambda a,b:{1,2}-set(a)=={1,2}-set(b)
M = powerset({1,2,3,4})
relate((relR,M))

NameError: name 'relate' is not defined

In [ ]:
# from collections import defaultdict, OrderedDict
# A_equiv_classes = OrderedDict()
# for Aa in sorted(A):
#     A_equiv_classes[Aa] = set()
#     for Ra, Rb in A_relation:
#         if Aa == Ra:
#             A_equiv_classes[Aa].add(Rb)
# dict(A_equiv_classes)
# # 0,0 | 0,0
# # 0,1 | 0,1
# # 0,2 | 0,2 1,1
# # 0,3 | 0,3 1,2
# # 0,4 | 0,4 1,3 2,2
# # 0,5 | 0,5 1,4 2,3
# # 1,5 | 1,5 2,4 3,3
# # 2,5 | 2,5 3,4
# # 3,5 | 3,5 4,4
# # 4,5 | 4,5
# # 5,5 | 5,5

In [ ]:
# relate_sqr(relate(lambda a,b: 0<=a<=b<=5, {0, 1,2,3,4, 5}))
# (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5),
# (1, 1), (1, 2), (1, 3), (1, 4), (1, 5),
# (2, 2), (2, 3), (2, 4), (2, 5),
# (3, 3), (3, 4), (3, 5),
# (4, 4), (4, 5),
# (5, 5)

In [15]:
def get_lambda_src(rel:R):
    try:
        fn_srcline:str=inspect.getsourcelines(rel)[0][0].strip()
    except TypeError:
        return rel.__qualname__
    if 'lambda' in fn_srcline:
        _,_,after_lam = map(str.strip,fn_srcline.partition('lambda'))
        lam_content,_,_=map(str.strip,after_lam.partition(' '))
        if ' ' in after_lam:
            return f'`lambda {lam_content[:-1]}`'
        else:
            return f'`lambda {lam_content}`'
    else:
        return fn_srcline

In [16]:
def randset(length, start=None, stop=None) -> set:
    """
    `start` is inclusive, `stop` is exclusive.
    >>> randset(10) == {-4, -3, -2, -1, 0, 1, 2, 3, 4}
    True

    >>> randset(5, 1, 4)
    {1, 2, 3}
    
    >>> 15 <= sum(randset(5, 1, 7)) <= 20
    True
    """
    if start is None and stop is None:
        if length % 2 == 0:
            length -= 1
        half = length // 2
        start = -half
        stop = half + 1
        tmp = set(range(start, stop))
        return tmp
    tmp = set()
    stop -= 1  # randint stop is inclusive
    tmp_len = len(tmp)
    while tmp_len < length and tmp_len < stop - start + 1:
        tmp.add(randint(start, stop))
        tmp_len = len(tmp)
    return tmp


In [17]:
def relate(rawrelation: RawRelation, *, quiet=True)->Prod:
    """Creates a relation between vec1 and vec2? by filtering their cartesian product.
    If only one vector is given, creates a relation between it and itself (vec1×vec1).
    Basically, `for x in A: for y in B: yield <x,y> if rel(x,y)`
    Returns a set of ordered pairs that's a subset of cart product.
    Examples:
    
    >>> V = {1, 2, 3}
    >>> relate(lambda a,b: a>b, V)
    {(3, 1), (3, 2), (2, 1)}
    
    >>> relate(gt, V) # same as example above
    {(3, 1), (3, 2), (2, 1)}
    
    >>> V_cart_prod = set(cartesian_prod(V))
    >>> relate(universal_rel, V) == V_cart_prod
    True
    """
    if quiet:
        qprint= lambda *args, **kwargs: True
    else:
        qprint = print
    if not is_raw_relation(rawrelation):
        if not is_product(rawrelation):
            raise ValueError(f"relate() | not a raw relation, not a product: {rawrelation}")
        print(f"WARN relate() | got a product, returning as is: {rawrelation}")
        return rawrelation
    relation = set()
    
    
    if len(rawrelation) == 3:
        fn, vec1, vec2 = rawrelation
    else:
        fn, vec1 = rawrelation
        vec2 = None
    fn_src=get_lambda_src(fn)
    if vec2 is None:
        sig_repr=f'relate({fn_src}, vec1 ({len(vec1)}) × vec1)'
            
        vec2=vec1
    else:
        
        sig_repr=f'relate({fn_src}, vec1 ({len(vec1)}) × vec2 (({len(vec2)}))'
            

    print(f'{sig_repr} |\n\t{vec1=}\n\t{vec2=}')
    
    for x,y in prod(vec1,vec2):
        if fn(x, y):
            relation.add((x, y))        
            qprint(f'\trelate() | added <x:{x}, y:{y}>')
    if quiet:
        print(f'{sig_repr} → {{ < , > ... }}({len(relation)})')
    else:
        print(f'{sig_repr} → {relation}')
    return relation

In [18]:
@overload
def multiply(relation: Relation,*, quiet=True)->Prod:
    """
    If (R:lambda, vec1, vec2), then R^2 of vec1 × vec2. 
    If (R:lambda, vec1), then R^2 of vec1 × vec1.
    If a single product, then product·product.
    """
    ...
@overload
def multiply(relation1: Relation, relation2:Relation,*, quiet=True)->Prod:
    """
    If (R1:lambda, vec1a, vec1b?), (R2:lambda, vec2a, vec2b?), then (R1 over vec1a×vec1b)·(R2 over vec2a×vec2b)
    if two products, then prod1 · prod2
    """
    ...

def multiply(relation1, relation2 = None, *, quiet=True)->Prod:
    """
    R1: A -> B, R2: B -> C; R1·R2 = {<a,c> ∈ A×C | ∃b ∈ B(aR1b ∧ bR2c)}
    
    Each pair from R1 (a,b) is compared against each pair from R2 (b,c).
    If a pair can shake hands (b1 == b2), then <a, c> is in the multipication result.
    
    >>> multiply({(2,3),(2,5)}, {(3,3),(5,9),(6,9)})
    ...
    {(2, 3), (2, 9)}
    
    >>> vec = {-1, 0, 1}
    >>> vec_cart = relate((universal_rel, vec))
    >>> just_vec = relate((identity_rel, vec))
    >>> multiply(vec_cart, just_vec) == vec_cart
    True
    """
    if is_raw_relation(relation1):
        R1 = relate(relation1)
        if relation2:
            R2 = relate(relation2)
            print(f'multiply | two pairs of (fn:R, vec:Vec), (fn:R, vec:Vec) overload | {R1=} | {R2=}')
        else:
            R2 = None
            print(f'multiply | one pair of (fn:R, vec:Vec) overload')
    else:
        R1 = relation1
        R2 = relation2
        print(f'multiply | products multiplication overload')
    
    mutiplied = set()
    if R2 is None:
        R2 = R1
    for a, b1 in R1:
        for b2, c in R2:
            if b1 == b2:
                mutiplied.add((a, c))
                print(f'multiply() | <\x1b[1ma:{a}\x1b[0m → b:{b1} → \x1b[1mc:{c}\x1b[0m>')
    return mutiplied

In [19]:
def inverse(A: Prod)->Prod:
    inverted = set()
    for a, b in A:
        inverted.add((b, a))
    return inverted


In [20]:
def relate_sqr(rel_or_product: Union[R, Prod], vec:Vec=None, quiet=True, alt=False):
    """Computes the squared relation.
    ⟨a,c⟩ ∈ 𝑹×𝑹: {⟨a,c⟩ | ∃b ∈ A (⟨a,b⟩ ∈ 𝑹 ∧ ⟨b,c⟩ ∈ 𝑹)}
    
    Forms:
    
    Returns a set of {} 
    >>> relate_sqr(lambda a,b:a>b, {1,2,3})
    {(3, 1)}
    
    
    >>> r2 = relate_sqr({(1, 1),(2, 2)})
    identity_rel
    
    Examples:
    
    >>> V = randset(30)
    >>> opposite_of_opposite = relate_sqr(opposite_rel, V)
    >>> I = relate(identity_rel, V)
    >>> I == opposite_of_opposite
    True

    
    
    """
    
    r2 = set()
    if callable(rel_or_product):
        rel = rel_or_product
        print(f'relate_sql({get_lambda_src(rel)}, vec:{vec})')
        # relate_sqr(lambda a,b:..., {1,2,3})
        if not alt:
            for a, c in cartesian_prod(vec):
                for b in vec:
                    if rel(a, b) and rel(b, c):
                        r2.add((a, c))
                        if not quiet:
                            print(f'relate_sqr() | added <a:{a}, c:{c}> (b:{b})')
        else:
            print(f'relate_sqr() | creating relation of {get_lambda_src(rel)} over vec={vec}')
            product:Prod = relate(rel, vec, quiet=quiet)
            print(f'relate_sqr() | multiplying {product} with itself')
            r2 = multiply(product, product, quiet=quiet) 
            print(f'relate_sqr() | multiply result = r2: {r2}')
            return r2
            #return relate_sqr(product, v, quiet=quiet, alt=False)
    
    else:
        # relate_sqr({<5,6>,<7,8>})
        # p111 / q15
        
        product = rel_or_product
        if not isinstance(next(iter(product)), tuple):
            raise TypeError((f"Can create a squared relation only with either a vector and a filter function, "
                             f"or with a product (set of ordered pairs).\n"
                            f"got: {product}"))
        print(f'relate_sql({product})\n\tmultiplying product * product')
        r2 = multiply(product, product, quiet=quiet)
        print(f'relate_sqr() | multiply result = r2: {r2}')
        if vec is not None:
            # ONLY FOR DEBUGGING, v has no use here
            try:
                known_rels = dict(
                        universal_rel=universal_rel,
                        empty_rel=empty_rel,
                        identity_rel=identity_rel,
                        opposite_rel=opposite_rel,
                        lower_equal=le,
                        lower_then=lt,
                        greater_equal=ge,
                        greater_then=gt,
                        not_equal=ne
                        )
                print(next(name for name, rel in known_rels.items() if r2 == relate(product, vec)))
            except StopIteration:
                pass
    return r2

In [21]:
def is_symmetric(rel: R, v1:Vec, quiet=True):
    """A relation is symmetric if for every pair <a,b> in R, also <b,a> in R
    In other words: rel(a,b) is True and rel(b,a) is True
    >>> symmetric = [universal_rel, empty_rel, identity_rel, opposite_rel, ne]
    >>> all(is_symmetric(_,randset(100)) for _ in symmetric)
    |R|: ...
    |R|: ...
    True
    
    >>> not_symmetric = [lt, le, gt, ge]
    >>> any(is_symmetric(_,randset(100)) for _ in not_symmetric)
    |R|: ...
    |R|: ...
    False
    
    """
    relation = relate(rel, v1, quiet=quiet)
    rel_len = len(relation)
    if not quiet:
        try:
            print(f'|R|: {rel_len} ({(rel_len / len((set(prod(v1, v1))))) * 100}%)')
        except ZeroDivisionError:
            pass
    for x, y in relation:
        if not rel(y, x):
            if not quiet:
                print(f'fail: rel(<x:{x}, y:{y}>) is True but rel(<y:{y}, x:{x}>) is False')
            return False
        if not quiet:
            print(f'good: rel(<x:{x}, y:{y}>) is True and rel(<y:{y}, x:{x}>) is True')
    return True

In [22]:
def is_anti_symmetric(rel: R, v1, v2=None, *, quiet=True):
    """A relation is anti symmetric if for every pair <a,b> in R, <b,a> is not in R
    In other words: rel(a,b) is True but rel(b,a) is False
    >>> anti_symmetric = [lt, gt, empty_rel] # also "⊆"
    >>> all(is_anti_symmetric(_,randset(100)) for _ in anti_symmetric)
    |R|: ...
    |R|: ...
    True
    
    >>> not_anti_symmetric = [universal_rel, identity_rel, opposite_rel, ne, le, ge, lambda a,b:b<a**2]
    >>> any(is_anti_symmetric(_,randset(100)) for _ in not_anti_symmetric)
    |R|: ...
    False
    """
    relation = relate(rel, v1, v2, quiet=quiet)
    rel_len = len(relation)
    print(f'|R|: {rel_len} ({(rel_len / len((set(prod(v1, v2 if v2 else v1))))) * 100}%)')
    for x, y in relation:
        if rel(y, x):
            if not quiet:
                print(f'fail: rel(<x:{x}, y:{y}>) is True but also rel(<y:{y}, x:{x}>) is True')
            return False
        if not quiet:
            print(f'good: rel(<x:{x}, y:{y}>) is True and rel(<y:{y}, x:{x}>) is False')
    return True


In [23]:
def is_reflexive(relation: Relation, *, quiet=True):
    """A relation is reflexive if every x in v satisfies rel(x,x)
    >>> reflexive = [universal_rel, identity_rel, le, ge]
    >>> all(is_reflexive(_,randset(100)) for _ in reflexive)
    ...
    True
    >>> not_reflexive = [ne, lt, gt, empty_rel, opposite_rel]
    >>> any(is_reflexive(_,randset(100)) for _ in not_reflexive)
    ...
    False
    
    """
    if is_raw_relation(relation):
        if len(relation) == 3:
            fn, vec1, vec2 = rawrelation
        else:
            fn, vec1 = rawrelation
            vec2 = None
    elif is_product(relation):
        raise NotImplementedError
    for x in v:
        if not rel(x, x):
            if not quiet:
                print(f'fail: rel(<{x},{x}>) is False')
            return False
        if not quiet:
            print(f'good: rel(<{x},{x}>) is True')
    return True

In [24]:
for reflexive in [universal_rel, identity_rel, le, ge]:
    vec = randset(4, -10, 30)
    # 1st way:
    for a in vec:
        if reflexive(a,a) is False:
            raise Exception
    
    # 2nd way:
    relation = relate((reflexive, vec))
    for a in vec:
        if (a,a) not in relation:
            raise Exception

relate(`lambda a,b:True`, vec1 (4) × vec1) |
	vec1={27, 28, -4, 6}
	vec2={27, 28, -4, 6}
relate(`lambda a,b:True`, vec1 (4) × vec1) → { < , > ... }(16)
relate(`lambda a,b:a==b`, vec1 (4) × vec1) |
	vec1={-8, 2, 26, 7}
	vec2={-8, 2, 26, 7}
relate(`lambda a,b:a==b`, vec1 (4) × vec1) → { < , > ... }(4)
relate(le, vec1 (4) × vec1) |
	vec1={-7, 10, 25, 6}
	vec2={-7, 10, 25, 6}
relate(le, vec1 (4) × vec1) → { < , > ... }(10)
relate(ge, vec1 (4) × vec1) |
	vec1={-5, 12, 20, 15}
	vec2={-5, 12, 20, 15}
relate(ge, vec1 (4) × vec1) → { < , > ... }(10)


In [25]:
# for not_reflexive in [ne, lt, gt, empty_rel, opposite_rel]:
#     vec = randset(3, -10, 30)
#     # 1st way:
#     for a in vec:
#         if not_reflexive(a,a):
#             raise Exception
    
#     # 2nd way:
#     relation = relate(not_reflexive, vec)
#     for a in vec:
#         if (a,a) in relation:
#             raise Exception

In [26]:
# for i in range(1,50):
#     mat = list(prod(randset(i, -5*i, i*10)))
#     if not is_symmetric(relR,mat):
#         raise Exception
#     if not is_reflexive(relR,mat):
#         raise Exception

In [27]:
# for i in range(1,50):
#     mat = list(prod(randset(i, -5*i, i*10)))
#     if not is_symmetric(relS,mat):
#         raise Exception
#     if not is_reflexive(relS,mat):
#         raise Exception

In [28]:
def is_anti_reflexive(rel: R, v, *, quiet=True):
    """A relation is anti reflexive if no x in v satisfies rel(x,x)
    >>> anti_reflexive = [ne, lt, gt, empty_rel]
    >>> all(is_anti_reflexive(_,randset(100)) for _ in anti_reflexive)
    ...
    True
    >>> not_anti_reflexive = [universal_rel, identity_rel, opposite_rel, le, ge]
    >>> any(is_anti_reflexive(_,randset(100)) for _ in not_anti_reflexive)
    ...
    False
    
    """
    for x in v:
        if rel(x, x):
            if not quiet:
                print(f'fail: rel(<{x},{x}>) is True')
            return False
        if not quiet:
            print(f'good: rel(<{x},{x}>) is False')
    return True

In [29]:
def is_empty_relation(rel: R, v1, v2=None, *, quiet=True):
    if v2 is None:
        product = set(prod(v1, v1))
    else:
        product = set(prod(v1, v2))
    for x, y in product:
        if rel(x, y) and (x, y) in product:
            return False
    return True

In [30]:
def is_universal_relation(rel: R, v1, v2=None, *, quiet=True):
    if v2 is None:
        product = set(prod(v1, v1))
    else:
        product = set(prod(v1, v2))
    for x, y in product:
        if rel(x, y) and (x, y) not in product:
            return False
    return True


In [31]:
def is_identity_relation(rel: Union[R, Prod], v1, v2=None, *, quiet=True):
    """
    >>> is_identity_relation(lambda a,b: a == b, {-1,0,1})
    True
    
    >>> is_identity_relation({(-1,-1), (0,0), (1,1)}, {-1,0,1})
    True
    
    >>> is_identity_relation({(-1,-1), (0,0), (1,1)}, {-1,0,1}, {-1,0,1,2})
    True
    """
    identity = relate(identity_rel, v1, v2, quiet=quiet)
    if callable(rel):
        relation = relate(rel, v1, v2, quiet=quiet)
        return identity == relation
    return identity == rel
